# BENCHMARK ALGORITHMS

### PARAMETERS

In [8]:
num_mat = 5
mat_size = 500
input_directory = "resources\input_matrices"
file_prefix = "SingMat"
benchmark_run = 1

Generate some matrices and save them to files.

In [12]:
import time as ti
import src.input.matrix_gen as gen

print(f"Generating {num_mat} singular matrices of size {mat_size} ...")

for i in range(1, num_mat+1):

    start_time = ti.time()
    mat = gen.non_singular_matrix(mat_size)
    end_time = ti.time()

    generation_time = end_time - start_time

    input_file_name = f"..\{input_directory}\{file_prefix}_{mat_size}_{i}.txt"

    file = open(input_file_name, "w")

    file.write(f"Matrix size: {mat_size}\n")
    file.write(f"Generation time (seconds): {generation_time}\n")

    for row in mat:
        file.write(f"{row}\n")

    file.close()

print(f"Saved {num_mat} matrices of size {mat_size} in '{input_directory}'.")

Generating 5 singular matrices of size 500 ...
Saved 5 matrices of size 500 in 'resources\input_matrices'.


Get relevant files and sort them.

In [14]:
import src.benchmarking.data_import as imp

files = imp.get_txt_files(f"..\{input_directory}", f"{file_prefix}_")
files.sort(key = lambda file_name: imp.extract_numbers(file_name, r'_\d+_', r'_\d+\.'))

print(files)

['SingMat_500_1.txt', 'SingMat_500_2.txt', 'SingMat_500_3.txt', 'SingMat_500_4.txt', 'SingMat_500_5.txt']


Read each file into a matrix, perform a benchmark and add the result to a file.


In [15]:
import src.input.bitfields as bitf
import src.algorithm.vanilla.cnot_synthesis_vanilla as vanilla
from copy import deepcopy

import csv

fieldnames = ['size', 'run', 'normal_time', 'optimized_time']

with open(rf'..\resources\benchmark_vanilla_{benchmark_run}.csv', mode='w', newline='') as benchmark_file:
    writer = csv.DictWriter(benchmark_file, fieldnames=fieldnames)
    writer.writeheader()

for file in files:

    size, run = imp.extract_numbers(file, r'_\d+_', r'_\d+\.')

    with open(rf"..\{input_directory}\{files[0]}", 'r') as input_file:
        lines = input_file.read().splitlines()

    matrix_norm = []
    for line in lines[2:]:
        row = bitf.to_bitfield(int(line), size)
        matrix_norm.append(row)

    matrix_opt = deepcopy(matrix_norm)

    print("Benchmarking for section size 5")
    start_time_norm = ti.time()
    _, circuit = vanilla.cnot_synthesis(matrix_norm, size, 5)
    end_time_norm = ti.time()
    benchmark_time_norm = end_time_norm - start_time_norm

    print("Benchmarking for section size 6")
    start_time_opt = ti.time()
    _, circuit = vanilla.cnot_synthesis(matrix_opt, size, 6)
    end_time_opt = ti.time()
    benchmark_time_opt = end_time_opt - start_time_opt

    with open(rf'..\resources\benchmark_vanilla_{benchmark_run}.csv', mode='a', newline='') as benchmark_file:
        writer = csv.DictWriter(benchmark_file, fieldnames=fieldnames)
        writer.writerow({'size': size, 'run': run, 'normal_time': benchmark_time_norm, 'optimized_time': benchmark_time_opt})

    #output_file_name = rf"..\resources\benchmark_vanilla_{benchmark_run}.txt"

    #file = open(output_file_name, "a")
    #file.write(f"size: {size}, run: {run}, normal_time: {benchmark_time_norm}, optimized_time: {benchmark_time_opt}\n")
    #file.close()

print("benchmark done")


Benchmarking for section size 5
Benchmarking for section size 6
Benchmarking for section size 5
Benchmarking for section size 6
Benchmarking for section size 5
Benchmarking for section size 6
Benchmarking for section size 5
Benchmarking for section size 6
Benchmarking for section size 5
Benchmarking for section size 6
benchmark done
